In [89]:
import keepa
import numpy as np
import requests
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd

In [2]:
with open("api_key.txt") as key_file :
    api_key = key_file.readline().strip()
api = keepa.Keepa(api_key)

In [37]:
# do a product search to filter out books that may be good to look at
book_params = {
    'deltaPercent30_USED_gte': 20,
    'deltaPercent30_USED_lte': 99,
    'deltaPercent90_COUNT_USED_gte': -50,
    'deltaPercent90_COUNT_USED_lte': 99,
    'avg180_SALES_gte': 0,
    'avg180_SALES_lte': 7000000,
    'rootCategory': 283155
}
# don't include list price because there's no way to compare the list price with the used price at this point in the process
# have Mark go through the other parameters and see if there's anything he thinks would also be helpful to include

In [102]:
test_params = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 10000,
    #"current_USED_gte": 0,
    #"current_USED_lte": 749,
    #"avg30_USED_gte": 1200,
    #"avg30_USED_lte": 1000000000,
    #"current_LISTPRICE_gte": 5000,
    #"current_LISTPRICE_lte": 100000000,
    #"avg180_LISTPRICE_gte": 5000,
    #"avg180_LISTPRICE_lte": 100000000,
    "rootCategory": 283155,
    "perPage": 10000
}

In [103]:
test3 = api.product_finder(test_params)

In [16]:
# now call the api to get the data on all these books
book_data = api.query(test[40:50])

100%|██████████| 10/10 [00:05<00:00,  1.94it/s]


In [33]:
book_data[1]['asin']

'0155510088'

In [167]:
# when it sold and how much it sold for
two_yrs_ago = datetime.datetime.now() - relativedelta(years=2)
sell_prices = pd.DataFrame()
sell_dates = pd.DataFrame()
for book in range(len(book_data)) :
    last_two_years = book_data[book]['data']['SALES_time'] > two_yrs_ago
    drop_dates_list = []
    sell_prices_list = []
    for day in np.where(last_two_years)[0] :
        if ((book_data[book]['data']['SALES'][day - 1] - 
            book_data[book]['data']['SALES'][day]) / 
            (book_data[book]['data']['SALES'][day - 1]) >= .04) :
            day_sold = book_data[book]['data']['SALES_time'][day]
            drop_dates_list.append(day_sold)
            try :
                day_sold_index = np.where(book_data[book]['data']['USED_time'] == day_sold)[0][0] - 1
            except IndexError :
                previous_days = np.where(book_data[book]['data']['USED_time'] < day_sold)
                day_sold_index = previous_days[0][-1]
            sell_prices_list.append(book_data[book]['data']['USED'][day_sold_index])
    drop_dates_df = pd.DataFrame({book_data[book]['asin']:drop_dates_list})
    sell_dates = pd.concat([sell_dates, drop_dates_df], ignore_index=True, axis=1)
    sell_prices_df = pd.DataFrame({book_data[book]['asin']:sell_prices_list})
    sell_prices = pd.concat([sell_prices, sell_prices_df], ignore_index=True, axis=1)

In [173]:
# find the peak of the 30 day sales average
sell_prices = sell_prices.fillna(0) #convert the NaNs to 0s so we can do math with them
rolling_averages = pd.DataFrame()
for book in range(len(book_data)) :
    two_yrs_ago = datetime.datetime.now() - relativedelta(years=2)
    two_yrs_30days = two_yrs_ago + datetime.timedelta(days=30)
    mving_window = [two_yrs_ago, two_yrs_30days]
    rolling_average = []
    for day in range(701) : # there's 701 days between the end of the moving window and today
        start = sell_dates[book] > mving_window[0]
        end = sell_dates[book] < mving_window[1]
        window = np.where(start & end)[0]
        rolling_average.append(np.average(sell_prices[book][window]))
        for date in range(len(mving_window)) :
            mving_window[date] += datetime.timedelta(days=1)
    rolling_averages[book] = rolling_average

In [174]:
rolling_averages

,0,1,2,3,4,5,6,7,8,9
0,NaN,2.40,2.770000,NaN,23.321500,8.808571,72.696471,21.99,41.406667,30.986667
1,NaN,2.40,2.741000,NaN,23.321500,8.611111,72.550000,21.99,39.948571,30.986667
2,NaN,2.40,2.741000,NaN,23.321905,8.611111,72.550000,21.99,39.948571,30.986667
3,NaN,2.40,2.741000,NaN,23.321053,8.611111,72.550000,21.99,39.948571,30.986667
4,NaN,2.40,2.741000,NaN,23.321053,8.611111,72.550000,21.99,39.948571,30.986667
...,...,...,...,...,...,...,...,...,...,...
696,NaN,NaN,16.473333,NaN,9.230000,11.730000,11.735000,13.97,15.420000,10.665000
697,NaN,25.59,16.473333,NaN,9.230000,15.000000,11.735000,NaN,11.525000,10.665000
698,NaN,25.59,16.473333,NaN,9.230000,15.000000,11.735000,NaN,11.525000,8.922000
699,7.26,25.59,16.473333,NaN,9.230000,15.000000,13.870000,9.61,11.525000,8.922000


In [154]:
sell_prices.head(20)

,0,1,2,3,4,5,6,7,8,9
0,9.57,2.40,5.15,4.41,23.33,9.94,71.26,21.99,41.75,30.50
1,4.97,2.40,2.38,1.95,23.33,8.52,71.33,20.00,41.33,30.50
2,7.01,2.99,2.02,0.00,23.33,8.52,63.17,20.00,41.29,30.50
3,6.92,2.99,2.48,0.00,23.33,8.78,61.92,18.11,41.24,30.49
4,2.42,2.99,2.74,0.00,23.33,8.78,61.88,12.77,41.06,30.49
5,1.98,2.99,2.74,0.00,23.33,8.78,61.88,12.74,41.77,30.49
6,5.00,2.99,2.74,0.00,23.33,8.34,61.84,10.12,31.20,30.49
7,5.00,2.99,2.73,0.00,23.33,7.92,65.30,10.10,31.20,30.49
8,5.00,4.41,1.95,0.00,23.33,7.92,75.99,21.00,31.20,34.93
9,5.00,5.31,2.48,0.00,23.33,5.40,79.81,20.98,30.56,34.93


In [105]:
sell_dates.head(20)

,0,1,2,3,4,5,6,7,8,9
0,2018-08-17 21:06:00,2018-05-11 20:08:00,2018-05-09 22:52:00,2019-05-04 08:56:00,2018-05-01 23:48:00,2018-05-07 21:54:00,2018-04-30 09:08:00,2018-05-13 19:08:00,2018-05-03 23:40:00,2018-05-05 22:24:00
1,2018-11-07 05:08:00,2018-10-12 19:18:00,2018-05-16 08:18:00,2020-04-27 19:16:00,2018-05-02 05:58:00,2018-05-15 00:26:00,2018-05-04 05:48:00,2018-06-21 19:16:00,2018-05-13 15:48:00,2018-05-06 07:48:00
2,2018-11-17 11:14:00,2018-12-18 17:16:00,2018-05-16 19:08:00,NaT,2018-05-04 00:20:00,2018-05-18 14:34:00,2018-05-15 15:56:00,2018-06-30 00:44:00,2018-05-15 15:30:00,2018-05-08 01:32:00
3,2018-11-28 19:52:00,2019-01-15 01:40:00,2018-05-17 12:48:00,NaT,2018-05-04 17:32:00,2018-05-23 03:56:00,2018-05-18 06:28:00,2018-08-08 17:12:00,2018-05-17 19:00:00,2018-05-15 22:08:00
4,2019-03-20 08:30:00,2019-02-04 06:42:00,2018-05-22 09:26:00,NaT,2018-05-09 10:18:00,2018-05-23 08:02:00,2018-05-18 21:20:00,2018-08-22 21:16:00,2018-05-19 09:44:00,2018-05-17 18:12:00
5,2019-03-28 12:32:00,2019-03-22 21:20:00,2018-05-22 19:52:00,NaT,2018-05-09 23:04:00,2018-05-23 18:56:00,2018-05-19 02:20:00,2018-08-23 00:36:00,2018-05-23 18:36:00,2018-05-18 15:00:00
6,2019-08-18 17:04:00,2019-04-01 08:24:00,2018-05-23 00:44:00,NaT,2018-05-11 22:02:00,2018-05-25 13:40:00,2018-05-19 09:48:00,2018-08-31 01:32:00,2018-05-29 15:56:00,2018-05-19 01:44:00
7,2019-08-25 03:48:00,2019-05-23 09:08:00,2018-05-23 23:04:00,NaT,2018-05-12 07:08:00,2018-05-30 08:40:00,2018-05-20 21:40:00,2018-09-01 15:48:00,2018-06-07 14:58:00,2018-05-21 20:40:00
8,2019-09-25 17:26:00,2019-12-03 04:20:00,2018-05-28 18:00:00,NaT,2018-05-13 23:36:00,2018-05-30 10:20:00,2018-05-21 18:30:00,2018-09-07 22:06:00,2018-06-09 14:32:00,2018-05-23 15:28:00
9,2019-10-08 03:44:00,2019-12-27 12:30:00,2018-05-30 08:48:00,NaT,2018-05-15 23:24:00,2018-06-05 05:42:00,2018-05-21 22:56:00,2018-09-08 18:56:00,2018-06-12 10:06:00,2018-06-29 09:34:00


In [29]:
2 year sales rank average 
sales rank average for selected dates

vv these two can try using and see if it improves the model. Can also switch them out for new price
average list price for selected dates
current list price

current used price
used avg for selected dates

(can later factor in used offer count)

True

In [61]:
# look at how the used price now compares to the used price normally
#valuation = pd.DataFrame()
diffs = {2018 : [], 2019 : []}
sales = {2018 : [], 2019 : []}
years = [2018,2019]
diff_avg = []
sales_avg = []
for book in range(len(book_data)) :
    used_current = book_data[book]['data']['USED'][-1]
    for year in years :
        start = book_data[book]['data']['USED_time'] > datetime.datetime(year, 8, 15, 0, 0)
        end = book_data[book]['data']['USED_time'] < datetime.datetime(year, 9, 16, 0, 0)
        high_tide = np.where(start & end)
        diff = (np.average(book_data[book]['data']['USED'][high_tide]) - used_current) / used_current
        diffs[year].append(diff)
        
        start = book_data[book]['data']['SALES_time'] > datetime.datetime(year, 8, 15, 0, 0)
        end = book_data[book]['data']['SALES_time'] < datetime.datetime(year, 9, 16, 0, 0)
        high_tide = np.where(start & end)
        sales[year].append(np.average(book_data[book]['data']['SALES'][high_tide]))
    diff_avg.append(np.average([diffs[2018][book],diffs[2019][book]], weights=[.4,.6]))
    sales_avg.append(np.average([diffs[2018][book],diffs[2019][book]]))

In [62]:
# look at the 2 year sales rank average
two_yrs_ago = datetime.datetime.now() - relativedelta(years=2)
two_yr_sales_avg = []
for book in range(len(book_data)) :
    within_2yrs = book_data[book]['data']['SALES_time'] > two_yrs_ago
    two_yr_sales_avg.append(np.average(book_data[book]['data']['SALES'][within_2yrs]))

In [63]:
diff_avg

[7.794544061302682,
 2.9476121562952247,
 12.605944391179293,
 12.136442141623487,
 1.7423847926267286,
 -0.02074380165289258,
 nan,
 33.48574444848927,
 1.6946377728566904,
 4.314431818181818]

In [24]:
# give each book a score
weights = [sales = .0000031, two_yr = .000001, diffs = 50]
scores = []
for book in range(len(book_data)) :
    metrics = [sales_avg[book], two_yr_sales_avg[book], diff_avg[book]]
    scores.append(np.dot(metrics, weights))

12